# `cfdm` - a Python reference implementation of the CF data model
-----

### https://ncas-cms.github.io/cfdm

### The `cfdm` package implements the CF data model for its internal data structures and so is able to process any CF-compliant dataset.

----
# The CF data model

<img src="field.png" style="height:600px"/>

-----
### It is not strict about CF-compliance, however, so that partially conformant datasets may be created in memory, ingested from existing datasets or written to new datasets.

### It is *not* an all-purpose analysis package - the main functionality is only that required to
* ### read field constructs from datasets
* ### write field constructs to datasets
* ### create, modify (including subspacing) and inspect field constructs in memory.

-----
# Import the package

In [1]:
import cfdm

----
# A simple field construct

In [2]:
!ncdump -h file0.nc

netcdf file0 {
dimensions:
	lat = 5 ;
	bounds2 = 2 ;
	lon = 8 ;
variables:
	double lat_bnds(lat, bounds2) ;
	double lat(lat) ;
		lat:units = "degrees_north" ;
		lat:standard_name = "latitude" ;
		lat:bounds = "lat_bnds" ;
	double lon_bnds(lon, bounds2) ;
	double lon(lon) ;
		lon:units = "degrees_east" ;
		lon:standard_name = "longitude" ;
		lon:bounds = "lon_bnds" ;
	double time ;
		time:units = "days since 2018-12-01" ;
		time:standard_name = "time" ;
	double q(lat, lon) ;
		q:project = "research" ;
		q:standard_name = "specific_humidity" ;
		q:units = "1" ;
		q:coordinates = "time" ;
		q:cell_methods = "area: mean" ;

// global attributes:
		:Conventions = "CF-1.8" ;
}


### Read the dataset

In [3]:
f = cfdm.read('file0.nc')[0]

### Have a summary look at the field construct

In [4]:
print(f)

Field: specific_humidity (ncvar%q)
----------------------------------
Data            : specific_humidity(latitude(5), longitude(8)) 1
Cell methods    : area: mean
Dimension coords: latitude(5) = [-75.0, ..., 75.0] degrees_north
                : longitude(8) = [22.5, ..., 337.5] degrees_east
                : time(1) = [2019-01-01 00:00:00]


### Have a look at the field construct's data

In [5]:
print(f.data.array)

[[0.007 0.034 0.003 0.014 0.018 0.037 0.024 0.029]
 [0.023 0.036 0.045 0.062 0.046 0.073 0.006 0.066]
 [0.11  0.131 0.124 0.146 0.087 0.103 0.057 0.011]
 [0.029 0.059 0.039 0.07  0.058 0.072 0.009 0.017]
 [0.006 0.036 0.019 0.035 0.018 0.037 0.034 0.013]]


### Have a comprehensive look 

In [6]:
f.dump()

----------------------------------
Field: specific_humidity (ncvar%q)
----------------------------------
Conventions = 'CF-1.8'
project = 'research'
standard_name = 'specific_humidity'
units = '1'

Data(latitude(5), longitude(8)) = [[0.007, ..., 0.013]] 1

Cell Method: area: mean

Domain Axis: latitude(5)
Domain Axis: longitude(8)
Domain Axis: time(1)

Dimension coordinate: latitude
    standard_name = 'latitude'
    units = 'degrees_north'
    Data(latitude(5)) = [-75.0, ..., 75.0] degrees_north
    Bounds:Data(latitude(5), 2) = [[-90.0, ..., 90.0]] degrees_north

Dimension coordinate: longitude
    standard_name = 'longitude'
    units = 'degrees_east'
    Data(longitude(8)) = [22.5, ..., 337.5] degrees_east
    Bounds:Data(longitude(8), 2) = [[0.0, ..., 360.0]] degrees_east

Dimension coordinate: time
    standard_name = 'time'
    units = 'days since 2018-12-01'
    Data(time(1)) = [2019-01-01 00:00:00]



----
# A field construct with every type of metadata construct

In [7]:
!ncdump -h file1.nc

netcdf file1 {
dimensions:
	atmosphere_hybrid_height_coordinate = 1 ;
	bounds2 = 2 ;
	y = 10 ;
	x = 9 ;
variables:
	double atmosphere_hybrid_height_coordinate_bounds(atmosphere_hybrid_height_coordinate, bounds2) ;
		atmosphere_hybrid_height_coordinate_bounds:formula_terms = "a: a_bounds b: b_bounds orog: surface_altitude" ;
	double atmosphere_hybrid_height_coordinate(atmosphere_hybrid_height_coordinate) ;
		atmosphere_hybrid_height_coordinate:computed_standard_name = "altitude" ;
		atmosphere_hybrid_height_coordinate:standard_name = "atmosphere_hybrid_height_coordinate" ;
		atmosphere_hybrid_height_coordinate:bounds = "atmosphere_hybrid_height_coordinate_bounds" ;
		atmosphere_hybrid_height_coordinate:formula_terms = "a: a b: b orog: surface_altitude" ;
	double y_bnds(y, bounds2) ;
	double y(y) ;
		y:units = "degrees" ;
		y:standard_name = "grid_latitude" ;
		y:bounds = "y_bnds" ;
	double x_bnds(x, bounds2) ;
	double x(x) ;
		x:units = "degrees" ;
		x:standard_name = "grid_longitude" ;

In [8]:
g = cfdm.read('file1.nc')[0]

In [9]:
print(g)

Field: air_temperature (ncvar%ta)
---------------------------------
Data            : air_temperature(atmosphere_hybrid_height_coordinate(1), grid_latitude(10), grid_longitude(9)) K
Cell methods    : grid_latitude(10): grid_longitude(9): mean where land (interval: 0.1 degrees) time(1): maximum
Field ancils    : air_temperature standard_error(grid_latitude(10), grid_longitude(9)) = [[0.76, ..., 0.32]] K
Dimension coords: atmosphere_hybrid_height_coordinate(1) = [1.5]
                : grid_latitude(10) = [2.2, ..., -1.76] degrees
                : grid_longitude(9) = [-4.7, ..., -1.18] degrees
                : time(1) = [2019-01-01 00:00:00]
Auxiliary coords: latitude(grid_latitude(10), grid_longitude(9)) = [[53.941, ..., 50.225]] degrees_N
                : longitude(grid_longitude(9), grid_latitude(10)) = [[2.004, ..., 8.156]] degrees_E
                : long_name=Grid latitude name(grid_latitude(10)) = [--, ..., b'kappa']
Cell measures   : measure:area(grid_longitude(9), grid_latitu

In [10]:
g.dump()

---------------------------------
Field: air_temperature (ncvar%ta)
---------------------------------
Conventions = 'CF-1.8'
project = 'research'
standard_name = 'air_temperature'
units = 'K'

Data(atmosphere_hybrid_height_coordinate(1), grid_latitude(10), grid_longitude(9)) = [[[262.8, ..., 269.7]]] K

Cell Method: grid_latitude(10): grid_longitude(9): mean where land (interval: 0.1 degrees)
Cell Method: time(1): maximum

Field Ancillary: air_temperature standard_error
    standard_name = 'air_temperature standard_error'
    units = 'K'
    Data(grid_latitude(10), grid_longitude(9)) = [[0.76, ..., 0.32]] K

Domain Axis: atmosphere_hybrid_height_coordinate(1)
Domain Axis: grid_latitude(10)
Domain Axis: grid_longitude(9)
Domain Axis: time(1)

Dimension coordinate: atmosphere_hybrid_height_coordinate
    computed_standard_name = 'altitude'
    standard_name = 'atmosphere_hybrid_height_coordinate'
    Data(atmosphere_hybrid_height_coordinate(1)) = [1.5]
    Bounds:Data(atmosphere_hybrid_h

-----
# Discrete sampling geometries

### Always represented by the CF data model as orthogonal mutlidimensional arrays, but may be read from and written to datasets as indexed and/or contiguous aarrays

### https://ncas-cms.github.io/cfdm/tutorial.html#discrete-sampling-geometries

In [11]:
!ncdump -h dsg.nc

netcdf dsg {
dimensions:
	station = 4 ;
	element = 24 ;
	bounds2 = 2 ;
variables:
	int64 count(station) ;
		count:sample_dimension = "element" ;
	double time_bounds(element, bounds2) ;
	double time(element) ;
		time:standard_name = "time" ;
		time:long_name = "time of measurement" ;
		time:units = "days since 1970-01-01 00:00:00" ;
		time:bounds = "time_bounds" ;
	double lat(station) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "station latitude" ;
		lat:units = "degrees_north" ;
	double lon(station) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "station longitude" ;
		lon:units = "degrees_east" ;
	double alt(station) ;
		alt:long_name = "vertical distance above the surface" ;
		alt:standard_name = "height" ;
		alt:units = "m" ;
		alt:positive = "up" ;
		alt:axis = "Z" ;
	string station_name(station) ;
		station_name:long_name = "station name" ;
		station_name:cf_role = "timeseries_id" ;
	int station_info(station) ;
		station_info:long_name = "station information" ;

In [12]:
h = cfdm.read('dsg.nc')[0]

In [13]:
print(h)

Field: precipitation_flux (ncvar%p)
-----------------------------------
Data            : precipitation_flux(cf_role=timeseries_id(4), ncdim%timeseries(9)) kg m-2 day-1
Auxiliary coords: time(cf_role=timeseries_id(4), ncdim%timeseries(9)) = [[1969-12-29 00:00:00, ..., 1970-01-07 00:00:00]]
                : latitude(cf_role=timeseries_id(4)) = [-9.0, ..., 78.0] degrees_north
                : longitude(cf_role=timeseries_id(4)) = [-23.0, ..., 178.0] degrees_east
                : height(cf_role=timeseries_id(4)) = [0.5, ..., 345.0] m
                : cf_role=timeseries_id(cf_role=timeseries_id(4)) = [b'station1', ..., b'station4']
                : long_name=station information(cf_role=timeseries_id(4)) = [-10, ..., -7]


In [14]:
print(h.data.array)

[[3.98 0.0 0.0 -- -- -- -- -- --]
 [0.0 0.0 0.0 3.4 0.0 0.0 4.61 -- --]
 [0.86 0.8 0.75 0.0 4.56 -- -- -- --]
 [0.0 0.09 0.0 0.91 2.96 1.14 3.86 0.0 0.0]]


### Write the field construct to a netCDF file 

In [15]:
cfdm.write(h, 'dsg_new.nc')

In [16]:
!ncdump -h dsg_new.nc

netcdf dsg_new {
dimensions:
	station = 4 ;
	element = 24 ;
	bounds2 = 2 ;
variables:
	int64 count(station) ;
		count:sample_dimension = "element" ;
	double time_bounds(element, bounds2) ;
	double time(element) ;
		time:standard_name = "time" ;
		time:long_name = "time of measurement" ;
		time:units = "days since 1970-01-01 00:00:00" ;
		time:bounds = "time_bounds" ;
	double lat(station) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "station latitude" ;
		lat:units = "degrees_north" ;
	double lon(station) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "station longitude" ;
		lon:units = "degrees_east" ;
	double alt(station) ;
		alt:long_name = "vertical distance above the surface" ;
		alt:standard_name = "height" ;
		alt:units = "m" ;
		alt:positive = "up" ;
		alt:axis = "Z" ;
	string station_name(station) ;
		station_name:long_name = "station name" ;
		station_name:cf_role = "timeseries_id" ;
	int station_info(station) ;
		station_info:long_name = "station informatio

In [17]:
h = h.compress('indexed')
cfdm.write(h, 'dsg_new_indexed.nc')
!ncdump -h dsg_new_indexed.nc

netcdf dsg_new_indexed {
dimensions:
	station = 4 ;
	sample = 24 ;
	bounds2 = 2 ;
variables:
	int64 index(sample) ;
		index:instance_dimension = "station" ;
	double time_bounds(sample, bounds2) ;
	double time(sample) ;
		time:standard_name = "time" ;
		time:long_name = "time of measurement" ;
		time:units = "days since 1970-01-01 00:00:00" ;
		time:bounds = "time_bounds" ;
	double lat(station) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "station latitude" ;
		lat:units = "degrees_north" ;
	double lon(station) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "station longitude" ;
		lon:units = "degrees_east" ;
	double alt(station) ;
		alt:long_name = "vertical distance above the surface" ;
		alt:standard_name = "height" ;
		alt:units = "m" ;
		alt:positive = "up" ;
		alt:axis = "Z" ;
	string station_name(station) ;
		station_name:long_name = "station name" ;
		station_name:cf_role = "timeseries_id" ;
	int station_info(station) ;
		station_info:long_name = "station info

In [18]:
h = h.uncompress()
cfdm.write(h, 'dsg_new_multidimensionsal.nc')
!ncdump -h dsg_new_multidimensionsal.nc

netcdf dsg_new_multidimensionsal {
dimensions:
	station = 4 ;
	timeseries = 9 ;
	bounds2 = 2 ;
variables:
	double time_bounds(station, timeseries, bounds2) ;
	double time(station, timeseries) ;
		time:standard_name = "time" ;
		time:long_name = "time of measurement" ;
		time:units = "days since 1970-01-01 00:00:00" ;
		time:bounds = "time_bounds" ;
	double lat(station) ;
		lat:standard_name = "latitude" ;
		lat:long_name = "station latitude" ;
		lat:units = "degrees_north" ;
	double lon(station) ;
		lon:standard_name = "longitude" ;
		lon:long_name = "station longitude" ;
		lon:units = "degrees_east" ;
	double alt(station) ;
		alt:long_name = "vertical distance above the surface" ;
		alt:standard_name = "height" ;
		alt:units = "m" ;
		alt:positive = "up" ;
		alt:axis = "Z" ;
	string station_name(station) ;
		station_name:long_name = "station name" ;
		station_name:cf_role = "timeseries_id" ;
	int station_info(station) ;
		station_info:long_name = "station information" ;
	double p(stat

-----
# Simple geometry cell bounds (since CF-1.8)

### Represented by the CF data model as bounds arrays with one or *two* trailing dimensions, but are read from and written to datasets using the CF-netCDF compression

### https://ncas-cms.github.io/cfdm/tutorial.html#geometry-cells

In [19]:
!ncdump -h geometry.nc

netcdf geometry {
dimensions:
	instance = 2 ;
	time = 4 ;
	bounds2 = 2 ;
	node = 13 ;
	part = 4 ;
variables:
	double time_bounds(time, bounds2) ;
		time_bounds:calendar = "gregorian" ;
	double time(time) ;
		time:standard_name = "time" ;
		time:units = "days since 2000-01-01" ;
		time:bounds = "time_bounds" ;
	double y(node) ;
		y:units = "degrees_north" ;
		y:standard_name = "latitude" ;
		y:axis = "Y" ;
	int64 node_count(instance) ;
	int64 part_node_count(part) ;
	int interior_ring(part) ;
	double lat(instance) ;
		lat:units = "degrees_north" ;
		lat:standard_name = "latitude" ;
		lat:nodes = "y" ;
	double x(node) ;
		x:units = "degrees_east" ;
		x:standard_name = "longitude" ;
		x:axis = "X" ;
	double lon(instance) ;
		lon:units = "degrees_east" ;
		lon:standard_name = "longitude" ;
		lon:nodes = "x" ;
	string instance_id(instance) ;
		instance_id:cf_role = "timeseries_id" ;
	double z(node) ;
		z:units = "m" ;
		z:standard_name = "altitude" ;
		z:axis = "Z" ;
	char datum ;
		datum:s

In [20]:
i = cfdm.read('geometry.nc')[0]

In [21]:
i.dump()

--------------------------------------
Field: precipitation_amount (ncvar%pr)
--------------------------------------
Conventions = 'CF-1.8'
featureType = 'timeSeries'
standard_name = 'precipitation_amount'
standard_units = 'kg m-2'

Data(cf_role=timeseries_id(2), time(4)) = [[1.0, ..., 8.0]]

Domain Axis: cf_role=timeseries_id(2)
Domain Axis: time(4)

Dimension coordinate: time
    standard_name = 'time'
    units = 'days since 2000-01-01'
    Data(time(4)) = [2000-01-16 12:00:00, ..., 2000-04-15 00:00:00]
    Bounds:calendar = 'gregorian'
    Bounds:Data(time(4), 2) = [[2000-01-01 00:00:00, ..., 2000-05-01 00:00:00]] gregorian

Auxiliary coordinate: latitude
    standard_name = 'latitude'
    units = 'degrees_north'
    Data(cf_role=timeseries_id(2)) = [25.0, 7.0] degrees_north
    Geometry: polygon
    Bounds:axis = 'Y'
    Bounds:standard_name = 'latitude'
    Bounds:units = 'degrees_north'
    Bounds:Data(cf_role=timeseries_id(2), 3, 4) = [[[0.0, ..., --]]] degrees_north
    Interi

In [22]:
y = i.construct('latitude')

In [23]:
y

<AuxiliaryCoordinate: latitude(2) degrees_north>

In [24]:
y.bounds

<Bounds: latitude(2, 3, 4) degrees_north>

In [25]:
print(y.bounds.data.array)

[[[0.0 15.0 0.0 --]
  [5.0 10.0 5.0 5.0]
  [20.0 35.0 20.0 --]]

 [[0.0 15.0 0.0 --]
  [-- -- -- --]
  [-- -- -- --]]]


In [26]:
cfdm.write(i, 'geometry_new.nc')

In [27]:
!ncdump -h geometry_new.nc

netcdf geometry_new {
dimensions:
	instance = 2 ;
	time = 4 ;
	bounds2 = 2 ;
	node = 13 ;
	part = 4 ;
variables:
	double time_bounds(time, bounds2) ;
		time_bounds:calendar = "gregorian" ;
	double time(time) ;
		time:standard_name = "time" ;
		time:units = "days since 2000-01-01" ;
		time:bounds = "time_bounds" ;
	double y(node) ;
		y:units = "degrees_north" ;
		y:standard_name = "latitude" ;
		y:axis = "Y" ;
	int64 node_count(instance) ;
	int64 part_node_count(part) ;
	int interior_ring(part) ;
	double lat(instance) ;
		lat:units = "degrees_north" ;
		lat:standard_name = "latitude" ;
		lat:nodes = "y" ;
	double x(node) ;
		x:units = "degrees_east" ;
		x:standard_name = "longitude" ;
		x:axis = "X" ;
	double lon(instance) ;
		lon:units = "degrees_east" ;
		lon:standard_name = "longitude" ;
		lon:nodes = "x" ;
	string instance_id(instance) ;
		instance_id:cf_role = "timeseries_id" ;
	double z(node) ;
		z:units = "m" ;
		z:standard_name = "altitude" ;
		z:axis = "Z" ;
	char datum ;
		dat

-----
### This representation and functionality is available to any library by *subclassing* `cfdm`


### `cf-python` (https://ncas-cms.github.io/cf-python)  does this, adding *metadata-aware* functions for
* ### subsetting
* ### weights calculations
* ### statistics
* ### regridding
* ### filtering
* ### *etc.*
-----